# Homeworke for week 8

In [6]:
# need to be run only once
# !wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
# and unzip it

In [7]:
# For Reproducibility purposes
import numpy as np
import torch

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [8]:
# Model
import torch.nn as nn

class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(3, 3), stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2)),
            nn.Flatten(),
            nn.Linear(32 * 99 * 99, 64),
            nn.ReLU(),
            nn.Linear(64, 1)  # logits output; use BCEWithLogitsLoss
        )

    def forward(self, x):
        return self.net(x)

model = SimpleCNN()
optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

# quick sanity check for the output shape
x = torch.randn(1, 3, 200, 200)
with torch.no_grad():
    y = model(x)
print("Output shape:", y.shape)

Output shape: torch.Size([1, 1])


In [9]:
# Q1
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 20073473


In [11]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )  # ImageNet normalization
])

train_dataset = datasets.ImageFolder(root="./data/train", transform=train_transforms)
validation_dataset = datasets.ImageFolder(root="./data/test", transform=train_transforms)

batch_size = 20
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

criterion = nn.BCEWithLogitsLoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [12]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)  # Ensure labels are float and shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.6325, Acc: 0.6462, Val Loss: 0.6152, Val Acc: 0.6318
Epoch 2/10, Loss: 0.5322, Acc: 0.7200, Val Loss: 0.6898, Val Acc: 0.6517
Epoch 2/10, Loss: 0.5322, Acc: 0.7200, Val Loss: 0.6898, Val Acc: 0.6517
Epoch 3/10, Loss: 0.4877, Acc: 0.7488, Val Loss: 0.6210, Val Acc: 0.6766
Epoch 3/10, Loss: 0.4877, Acc: 0.7488, Val Loss: 0.6210, Val Acc: 0.6766
Epoch 4/10, Loss: 0.4419, Acc: 0.7925, Val Loss: 0.7786, Val Acc: 0.6219
Epoch 4/10, Loss: 0.4419, Acc: 0.7925, Val Loss: 0.7786, Val Acc: 0.6219
Epoch 5/10, Loss: 0.3772, Acc: 0.8275, Val Loss: 0.6921, Val Acc: 0.6517
Epoch 5/10, Loss: 0.3772, Acc: 0.8275, Val Loss: 0.6921, Val Acc: 0.6517
Epoch 6/10, Loss: 0.3170, Acc: 0.8775, Val Loss: 0.6288, Val Acc: 0.6667
Epoch 6/10, Loss: 0.3170, Acc: 0.8775, Val Loss: 0.6288, Val Acc: 0.6667
Epoch 7/10, Loss: 0.2242, Acc: 0.9213, Val Loss: 0.7828, Val Acc: 0.6965
Epoch 7/10, Loss: 0.2242, Acc: 0.9213, Val Loss: 0.7828, Val Acc: 0.6965
Epoch 8/10, Loss: 0.2906, Acc: 0.8950, Val Loss: 0.

In [15]:
# Q3 and Q4

train_acc_median = float(np.median(history['acc']))
train_loss_std = float(np.std(history['loss']))
print('Median train accuracy:', train_acc_median)
print('Std dev train loss:', train_loss_std)

Median train accuracy: 0.8525
Std dev train loss: 0.15926927282896963


In [16]:
# Augmented pipeline
aug_train_transforms = transforms.Compose([
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

aug_val_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

aug_train_dataset = datasets.ImageFolder(root="./data/train", transform=aug_train_transforms)
aug_validation_dataset = datasets.ImageFolder(root="./data/test", transform=aug_val_transforms)

aug_batch_size = 20
aug_train_loader = DataLoader(aug_train_dataset, batch_size=aug_batch_size, shuffle=True)
aug_validation_loader = DataLoader(aug_validation_dataset, batch_size=aug_batch_size, shuffle=False)


In [17]:
# Continue training with augmentations for 10 more epochs
aug_num_epochs = 10
aug_history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(aug_num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in aug_train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(aug_train_dataset)
    epoch_acc = correct_train / total_train
    aug_history['loss'].append(epoch_loss)
    aug_history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in aug_validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(aug_validation_dataset)
    val_epoch_acc = correct_val / total_val
    aug_history['val_loss'].append(val_epoch_loss)
    aug_history['val_acc'].append(val_epoch_acc)

    print(f"Aug Epoch {epoch+1}/{aug_num_epochs}, Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Aug Epoch 1/10, Loss: 0.6820, Acc: 0.6412, Val Loss: 0.6229, Val Acc: 0.6915
Aug Epoch 2/10, Loss: 0.5621, Acc: 0.6813, Val Loss: 0.6245, Val Acc: 0.6866
Aug Epoch 2/10, Loss: 0.5621, Acc: 0.6813, Val Loss: 0.6245, Val Acc: 0.6866
Aug Epoch 3/10, Loss: 0.5255, Acc: 0.7175, Val Loss: 0.6020, Val Acc: 0.7313
Aug Epoch 3/10, Loss: 0.5255, Acc: 0.7175, Val Loss: 0.6020, Val Acc: 0.7313
Aug Epoch 4/10, Loss: 0.5241, Acc: 0.7350, Val Loss: 0.7668, Val Acc: 0.6517
Aug Epoch 4/10, Loss: 0.5241, Acc: 0.7350, Val Loss: 0.7668, Val Acc: 0.6517
Aug Epoch 5/10, Loss: 0.5264, Acc: 0.7388, Val Loss: 0.5955, Val Acc: 0.7164
Aug Epoch 5/10, Loss: 0.5264, Acc: 0.7388, Val Loss: 0.5955, Val Acc: 0.7164
Aug Epoch 6/10, Loss: 0.4822, Acc: 0.7800, Val Loss: 0.5848, Val Acc: 0.7214
Aug Epoch 6/10, Loss: 0.4822, Acc: 0.7800, Val Loss: 0.5848, Val Acc: 0.7214
Aug Epoch 7/10, Loss: 0.4717, Acc: 0.7638, Val Loss: 0.6868, Val Acc: 0.6816
Aug Epoch 7/10, Loss: 0.4717, Acc: 0.7638, Val Loss: 0.6868, Val Acc: 0.6816

In [19]:
# Q5 and Q6
mean_test_loss = float(np.mean(aug_history['val_loss']))
last5_val_acc = float(np.mean(aug_history['val_acc'][-5:]))
print('Mean test loss across aug epochs:', mean_test_loss)
print('Avg test accuracy (epochs 6-10) with aug:', last5_val_acc)

Mean test loss across aug epochs: 0.6309545061184992
Avg test accuracy (epochs 6-10) with aug: 0.7104477611940299
